In [ ]:
import torch
import torch.nn as nn

import numpy as np

## MNIST 데이터셋 불러오기

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

토치비전이라는 라이브러리 , 이미지 영상처리에 사용되는 라이브러리이다. 

In [ ]:
mnist_train = datasets.MNIST(root='MNIST_data/',
                             train=True,
                             transform=transforms.ToTensor(),
                             download=True)

mnist_test = datasets.MNIST(root='MNIST_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

데이터셋 읽어오는 과정

In [ ]:
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data/
    Split: Test
    StandardTransform
Transform: ToTensor()

각각 트레인 테스트 정보들

In [ ]:
mnist_train.data.shape

torch.Size([60000, 28, 28])

데이터의 차원을 항상 신경써야한다. 

In [ ]:
input_dim = mnist_train.data.shape[1] * mnist_train.data.shape[2]
output_dim = 10

뒷쪽에 모델을 정의할때 사용할려고 만든것임. input 디비전의 크리를 생각해보면 쭉펴서 풀리커넥티드에 넣는다고 생각해도면 리니어하게 되어있으니 28*28만큼 들어감 

In [ ]:
train_loader = DataLoader(dataset=mnist_train, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=mnist_test, batch_size=100, shuffle=False)

## 다층 퍼셉트론으로 MNIST 손글씨 분류하기

In [ ]:
tmp = torch.randn((100, 1, 28, 28))
tmp.shape

torch.Size([100, 1, 28, 28])

진행할때 꼭 필요한 부분은 아니나 차원에 대한 이해를 위해서 임의의 난수를 만듬.
view 함수 설명하기 위해서 있는 것임

In [ ]:
tmp = tmp.view(tmp.size(0), -1)
tmp.shape

torch.Size([100, 784])

view라는 함수는 해당하는 차원으로 tmp의 형태를 바꿔준다. 원소의 값을 유지시키나 차원의 형태를 바꿔줌 . 맨처음에 있는 차원은 첫번째 차원은 그대로 가져가고 -1라는  의미는 알아서 해달라는 의미이다. 

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(input_dim,100)
        self.fc2 = nn.Linear(100,100)
        self.fc3 = nn.Linear(100,output_dim)
        
        self.relu = nn.ReLU()
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

MLP_classifier = MLP(input_dim, output_dim)

인풋dim 아우풋dim를 mlp만들때 사용한다. 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(MLP_classifier.parameters(), lr=1e-2)

In [ ]:
for epoch in range(10):
    for X, y in train_loader:
        optimizer.zero_grad()
        X = X.view(X.size(0), -1)
        y_pred = MLP_classifier(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

    print("loss of {} epoch: {}".format(epoch, loss.item()))

loss of 0 epoch: 0.12353382259607315
loss of 1 epoch: 0.12012302130460739
loss of 2 epoch: 0.0906856432557106
loss of 3 epoch: 0.12730292975902557
loss of 4 epoch: 0.15605437755584717
loss of 5 epoch: 0.05139518156647682
loss of 6 epoch: 0.0417679063975811
loss of 7 epoch: 0.010552599094808102
loss of 8 epoch: 0.053344909101724625
loss of 9 epoch: 0.15670929849147797


view들어간 이유는 차원을 바꿔주는 효과 -> 배치사이즈를 바꿔주니 우리가 구성한 네트워크는 mlp이니까 2차원 인풋을 받아드리지 못함 그래서 그것을 벡터형태로 쭉 펼쳐서 바꿔줌 

In [ ]:
MLP_classifier.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for X_test, y in test_loader:
        X_test = X_test.view(X_test.size(0), -1)
        y_pred = MLP_classifier(X_test)
        _, predicted = torch.max(y_pred.data, 1)
        correct += predicted.eq(y.data.view_as(predicted)).sum()

    # 정확도 출력
    data_num = len(test_loader.dataset)  # 데이터 총 건수
    print('\n테스트 데이터 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))


테스트 데이터 예측 정확도: 9714/10000 (97%)



모델이 받아들이는게 xtest로 똑같이 view를 통해서 바꿔서 예측 해줘야한다.

분류를 할려면 0~9중 하나 고르는 과정 _, predicted = torch.max(y_pred.data, 1)


## CNN으로 MNIST 손글씨 분류하기

- 1번 레이어 : 합성곱층(Convolutional layer)
 - 합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
 - 맥스풀링(kernel_size=2, stride=2))

- 2번 레이어 : 합성곱층(Convolutional layer)
 - 합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
 - 맥스풀링(kernel_size=2, stride=2))

- 3번 레이어 : 전결합층(Fully-Connected layer)
 - 전결합층 (입력 **????** / 출력 10) + 활성화 함수 Softmax

In [ ]:
tmp = torch.randn((100, 1, 28, 28))
conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

In [ ]:
tmp = conv1(tmp)
tmp.shape

torch.Size([100, 32, 28, 28])

아웃채널이 32라서 32

In [ ]:
tmp = maxpool(tmp)
tmp.shape

torch.Size([100, 32, 14, 14])

채널 갯수는 동일한데 크기가 줄어듬 즉 이미지의 크기가 줄어든다

In [ ]:
tmp = conv2(tmp)
tmp.shape

torch.Size([100, 64, 14, 14])

In [ ]:
tmp = maxpool(tmp)
tmp.shape

torch.Size([100, 64, 7, 7])

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc = torch.nn.Linear(7 * 7 * 64, 10)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
  
CNN_classifier = CNN()

maxpool로 차원을 늘려줌


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNN_classifier.parameters(), lr=1e-2)

In [ ]:
for epoch in range(3):
    for X, y in train_loader:
        optimizer.zero_grad()
        y_pred = CNN_classifier(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

    print("loss of {} epoch: {}".format(epoch, loss.item()))

loss of 0 epoch: 0.24180564284324646
loss of 1 epoch: 0.06301598250865936
loss of 2 epoch: 0.10591074824333191


여기는 바로 이미지 처리르 하기때문에 펴주는작없이 필요없다(view) 컨볼루션 layer에서 바로 나오기때문이다. 로스를 최소화 하는 방향으로 파라미터들이 계속 업데이트되면서 앞에는 1자로 펼쳐서 넣어줘야하고 이것은 그럴 필요가 없다.


In [ ]:
CNN_classifier.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for X_test, y in test_loader:
        y_pred = CNN_classifier(X_test)
        _, predicted = torch.max(y_pred.data, 1)
        correct += predicted.eq(y.data.view_as(predicted)).sum()

    # 정확도 출력
    data_num = len(test_loader.dataset)  # 데이터 총 건수
    print('\n테스트 데이터 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))


테스트 데이터 예측 정확도: 9609/10000 (96%)



마지막에 fuul connected layer를 통해서 